In [1]:
from Levenshtein import distance
import numpy as np
import pandas as pd
import os
import unicodedata
import re

In [2]:
def get_levenshtein(x,y):
    # ordering the inputs by length so that a<=b
    a=max(len(x), len(y))
    b=min(len(x), len(y))
    if a==0: # one of the input is empty
        return 1.
    if a > b+500: # if there are more than 500 more chars then return 100%
        return 1.
    # return the levenshtein distance normalized by the mean length
    mean_length = (a + b)/2.
    return 2 * distance(x,y)/mean_length

def normalize_text(x):
    return unicodedata.normalize("NFKD", x.replace(";"," ")).upper()

In [3]:
def keep_digits(x):
    return ''.join(filter(lambda x: x.isdigit(), x))

In [4]:
def lecture_amendement(folder, projet_loi, start_offset):
    res=""
    starts=[]
    row=0
    all_amendement = []
    print("Lecture des fichiers dans {} : ".format(folder))
    for f in os.listdir(folder):
        print("{}".format(f), end=',')
        current_file= open(folder+"/"+f, encoding="iso-8859-1")
        
        for line in current_file:
            all_amendement.append(normalize_text(line.strip()))
            if normalize_text(line.strip()[0:len(projet_loi)]) == normalize_text(projet_loi):
                starts.append(row-start_offset)
            row+=1
                
        current_file.close()
    print("")
    print("")
    return starts, all_amendement


def get_amendement_nb(x, nb_offset = 0):
    for i in range(0, len(x)):
        if x[i][0:3]=="N° ":
            return x[i].replace("N° ", "")
    return "numéro amendement introuvable 0"
#    try:
#        return x[nb_offset].replace("N° ","")
#    except:
#        return "0"


def get_amendement_text(x, start_signal, start_offset):
    start,end=0,0
    for i in range(0, len(x)):
        if x[i]==normalize_text(start_signal):
            start=i+start_offset
            
        if i<len(x)-3 and len(x[i])==0 and len(x[i+1])==0 and len(x[i+2])==0:
            end=i
        
        if len(x[i])<18 and ((normalize_text(x[i][0:5])==normalize_text("EXPOS")) or i==len(x)-1 \
        or (normalize_text(x[i][0:5])==normalize_text("Objet"))):
            end=i
            break
    candidate = x[start:end]
    res=[]
    for i in range(0, len(candidate)):
        if normalize_text("Cet amendement est en cours") in normalize_text(candidate[i]):
            continue
        elif len(candidate[i]) < 2:
            continue
        else:
            res.append(candidate[i])
    return " ".join(res).replace("  ", " ")


def get_amendement_art(x, start_signal):
    has_started=False
    for i in range(0, len(x)):
        if x[i]==normalize_text(start_signal):
            has_started = True
            
        if(has_started) and 'ARTICLE' in normalize_text(x[i]):
            return re.sub(".*ARTICLE ","",x[i])
    return ""
            

def get_amendements(starts, all_amendements, version, start_signal, start_offset):
    amendements_nb, amendements_txt, amendements_art, begins, ends = [],[],[],[],[]
    for i in range(0, len(starts)):
        if i<len(starts)-1:
            begin=starts[i]
            end=starts[i+1]
        else:
            begin=starts[-1]
            end=len(all_amendements)
        #print(begin, end)
        current_amendement = all_amendements[begin:end]
        #print(starts[i], starts[i+1])
        amendements_nb.append(get_amendement_nb(current_amendement))
        amendements_txt.append(get_amendement_text(current_amendement, start_signal, start_offset))
        amendements_art.append(get_amendement_art(current_amendement, start_signal))
        begins.append(begin)
        ends.append(end)
    
    df_amendement = pd.DataFrame({'nb':amendements_nb, \
                                  'txt':amendements_txt, \
                                  'article':amendements_art, 'begin':begins, 'end':ends})
    print("Il y a {} amendements dans la liasse {}".format(len(df_amendement), version))
    return df_amendement


In [5]:
def build_amendement(path, parameters):
    starts, all_amendement= lecture_amendement(path, parameters["projet_loi"], parameters["nb_line_before_projet_loi"])
    df_amendements = get_amendements(starts, all_amendement,path,\
                                     parameters["separateur"], \
                                     parameters["nb_line_txt_begins_after_sep"])
    return df_amendements

In [6]:
def compute_match(df_amendements_1, df_amendements_2, version1, version2):
    
    threshold = 0.01

    corresp={}

    for i1, row1 in df_amendements_1.iterrows():

        if i1 % 20 == 0:
            print( "{} %".format(int(i1 * 100 / len(df_amendements_1))), end="  ")
        for i2,row2 in df_amendements_2.iterrows():
            l = get_levenshtein(row1.txt,row2.txt)
            current_similarity = 1-l

            if(current_similarity > threshold):

                update = True
                if row2.nb in corresp:
                    previous_similarity = corresp[row2.nb][3]
                    if previous_similarity >= current_similarity:
                        update = False

                if update:
                    corresp[row2.nb] = (row1.nb, row1.txt, row2.txt, current_similarity, row2.article)
                    
    amendement_id2, amendement_id1, amendement_txt2, amendement_txt1, similarity, article2 \
    = [], [], [], [], [], []

    for id2 in corresp:
        id1, txt1, txt2, simi, art2 = corresp[id2] 
        amendement_id2.append(str(id2))
        amendement_id1.append(str(id1))
        amendement_txt2.append(txt2)
        article2.append(art2)
        amendement_txt1.append(txt1)
        similarity.append("{0:.0%}".format(simi))

    df_amendements_matched = pd.DataFrame( { \
                                         "N° "+version1:amendement_id1, \
                                         "N° "+version2:amendement_id2, \
                                         "Amendement "+version1:amendement_txt1, \
                                         "Amendement "+version2:amendement_txt2, \
                                         "Article "+version2:article2, \
                                         "Similarité":similarity })
    print("Il y a {} amendements communs".format(len(df_amendements_matched)))
    
    df_amendements_matched["id2"] = df_amendements_matched["N° "+version2]\
    .apply(lambda x:keep_digits(x))\
    .astype(int)
    df_amendements_matched.sort_values(by="id2", inplace = True, ascending=True)
    del df_amendements_matched["id2"]
    
    
    nb_id2_matched = len(df_amendements_matched["N° "+version2].unique())
    nb_id1_matched = len(df_amendements_matched["N° "+version1].unique())
    nb_id2 = len(df_amendements_2)
    nb_id1 = len(df_amendements_1)
    print("""Sur les {} amendements de la nouvelle liasse, {} ont été retrouvés dans la première liasse, \
    soit {:.0%}.
    Sur les {} amendements de l'ancienne liasse, {} ont été ré-utilisés dans la seconde liasse, \
    soit {:.0%}.
          """\
         .format(nb_id2, nb_id2_matched, nb_id2_matched/nb_id2, nb_id1, nb_id1_matched, nb_id1_matched/nb_id1))
    
    return df_amendements_matched

In [7]:
def match_amendement(df_amendements_2, df_amendements_matched, version1, version2):
    
    df_results = pd.merge(df_amendements_2, df_amendements_matched, left_on = "nb", right_on="N° "+version2)

    df_results["N° "+version2] = df_results["nb"]
    df_results["Amendement "+version2] = df_results["txt"]

    del df_results["begin"], df_results["end"], df_results["nb"], df_results["txt"], df_results["article"]

    df_results["N° "+version1].fillna("non trouvé", inplace = True)
    df_results["Amendement "+version1].fillna("non trouvé", inplace = True)
    df_results["Similarité"].fillna("0%", inplace = True)
    
    df_results.to_excel("matching_{}_{}.xlsx".format(version1, version2), index=False)
    
    return df_results

In [8]:
params = {}
params["AN"] = {}
params["AN"]["projet_loi"] = normalize_text("EQUILIBRE DANS LE SECTEUR AGRICOLE ET ALIMENTAIRE")
params["AN"]["nb_line_before_projet_loi"] = 5
#params["AN"]["nb_line_numero_amendement"] = 1
params["AN"]["separateur"] = "----------"
params["AN"]["nb_line_txt_begins_after_sep"] = 3

df_amendements_1 = build_amendement("AN/", params["AN"])  
df_amendements_1.head()

Lecture des fichiers dans AN/ : 
10_ apräs 15.txt,11_art 15 bis Ö avant 16.txt,12- art 16 Ö la fin.txt,1_  avant 11 et 11 liasse 2.txt,2 _ art 11- apräs 11.txt,3_ 11 bis apräs 11 quindecies.txt,4- art 11 sexdecies -apräs 12.txt,5_ art 12 bis -apräs 13 ter.txt,6_ apräs 13 ter- apräs 14.txt,8_ 14 bis -apräs 14 undecies.txt,9_ art 15.txt,

Il y a 1379 amendements dans la liasse AN/


,article,begin,end,nb,txt
0,ADDITIONNEL,0,33,249,APRÈS LE II DE LARTICLE L. 211-1 DU CODE DE ...
1,ADDITIONNEL,33,63,450,LE 9° DU II DE LARTICLE L. 110-1 DU CODE DE L...
2,ADDITIONNEL,63,93,444,AU IV DE LARTICLE L. 122-1 DU CODE DE LENVIR...
3,ADDITIONNEL,93,125,453,APRÈS LE V DE LARTICLE L. 122-1 DU CODE DE L...
4,ADDITIONNEL,125,154,451,LES ARTICLES L. 181-7 ET L. 514-6 DU CODE DE L...


In [9]:
params["senat"] = {}
params["senat"]["projet_loi"] = normalize_text("Relations commerciales dans le secteur agricole et alimentaire")
params["senat"]["nb_line_before_projet_loi"] = 2
#params["senat"]["nb_line_numero_amendement"] = 5
params["senat"]["separateur"] = "présenté par"
params["senat"]["nb_line_txt_begins_after_sep"] = 9

df_amendements_2 = build_amendement("senat/", params["senat"])  
df_amendements_2.head()

Lecture des fichiers dans senat/ : 
.DS_Store,liasse sénat 1- 362 COMECO.txt,

Il y a 259 amendements dans la liasse senat/


,article,begin,end,nb,txt
0,15,3,39,COM-1,INSÉRER UN ARTICLE ADDITIONNEL AINSI RÉDIGÉ...
1,15 BIS (NOUVEAU),39,69,COM-2,INSÉRER UN ARTICLE ADDITIONNEL AINSI RÉDIGÉ...
2,11 SEXIES (NOUVEAU),69,123,COM-3,INSÉRER UN ARTICLE ADDITIONNEL AINSI RÉDIGÉ...
3,14 QUATER (NOUVEAU),123,149,COM-4,INSÉRER UN ARTICLE ADDITIONNEL AINSI RÉDIGÉ...
4,14 QUATER (NOUVEAU),149,183,COM-5,INSÉRER UN ARTICLE ADDITIONNEL AINSI RÉDIGÉ...


mot="CUNICOLE"

df_amendements_1["cont"] = df_amendements_1["txt"].apply(lambda x:mot in x)

df_amendements_2["cont"] = df_amendements_2["txt"].apply(lambda x:mot in x)

df_amendements_1[df_amendements_1.cont==1]
df_amendements_2[df_amendements_2.cont==1]
del df_amendements_1["cont"]
del df_amendements_2["cont"]

In [10]:
df_amendements_matched = compute_match(df_amendements_1, df_amendements_2, "AN", "senat")
df_amendements_matched.head()

0 %  1 %  2 %  4 %  5 %  7 %  8 %  10 %  11 %  13 %  14 %  15 %  17 %  18 %  20 %  21 %  23 %  24 %  26 %  27 %  29 %  30 %  31 %  33 %  34 %  36 %  37 %  39 %  40 %  42 %  43 %  44 %  46 %  47 %  49 %  50 %  52 %  53 %  55 %  56 %  58 %  59 %  60 %  62 %  63 %  65 %  66 %  68 %  69 %  71 %  72 %  73 %  75 %  76 %  78 %  79 %  81 %  82 %  84 %  85 %  87 %  88 %  89 %  91 %  92 %  94 %  95 %  97 %  98 %  Il y a 110 amendements communs
Sur les 259 amendements de la nouvelle liasse, 110 ont été retrouvés dans la première liasse,     soit 42%.
    Sur les 1379 amendements de l'ancienne liasse, 87 ont été ré-utilisés dans la seconde liasse,     soit 6%.
          


,Amendement AN,Amendement senat,Article senat,N° AN,N° senat,Similarité
70,A LA FIN DU I. DE L'ARTICLE L 251-1 DU CODE RU...,INSÉRER UN ARTICLE ADDITIONNEL AINSI RÉDIGÉ...,14 QUATER (NOUVEAU),2058,COM-4,62%
2,"APRÈS L'ARTICLE 15, INSÉRER L'ARTICLE SUIVAN...",APRÈS L'ARTICLE 10 OCTIES (NOUVEAU) INSÉRER ...,10 OCTIES (NOUVEAU),76,COM-16,82%
14,LA SECTION 2 DU CHAPITRE IER DU TITRE IER DU L...,AVANT L'ARTICLE 16 A (NOUVEAU) INSÉRER UN ART...,16 A (NOUVEAU),499,COM-18,8%
27,"« À PARTIR DUN CADRE NATIONAL, LES RÉGIONS ...",APRÈS L'ARTICLE 16 A (NOUVEAU) INSÉRER UN AR...,16 A (NOUVEAU),2317,COM-19,26%
55,LE TITRE II DU LIVRE V DE LA PARTIE LÉGISLATI...,APRÈS L'ARTICLE 11 QUINDECIES (NOUVEAU) INSÉ...,11 QUINDECIES (NOUVEAU),2131,COM-22,56%


In [12]:
df_results = match_amendement(df_amendements_2, df_amendements_matched, "AN", "senat")
df_results.head()

,Amendement AN,Amendement senat,Article senat,N° AN,N° senat,Similarité
0,A LA FIN DU I. DE L'ARTICLE L 251-1 DU CODE RU...,INSÉRER UN ARTICLE ADDITIONNEL AINSI RÉDIGÉ...,14 QUATER (NOUVEAU),2058,COM-4,62%
1,"APRÈS L'ARTICLE 15, INSÉRER L'ARTICLE SUIVAN...",APRÈS L'ARTICLE 10 OCTIES (NOUVEAU) INSÉRER ...,10 OCTIES (NOUVEAU),76,COM-16,82%
2,LA SECTION 2 DU CHAPITRE IER DU TITRE IER DU L...,AVANT L'ARTICLE 16 A (NOUVEAU) INSÉRER UN ART...,16 A (NOUVEAU),499,COM-18,8%
3,"« À PARTIR DUN CADRE NATIONAL, LES RÉGIONS ...",APRÈS L'ARTICLE 16 A (NOUVEAU) INSÉRER UN AR...,16 A (NOUVEAU),2317,COM-19,26%
4,LE TITRE II DU LIVRE V DE LA PARTIE LÉGISLATI...,APRÈS L'ARTICLE 11 QUINDECIES (NOUVEAU) INSÉ...,11 QUINDECIES (NOUVEAU),2131,COM-22,56%


In [ ]:
#%matplotlib inline
#df_amendements_matched["Simi"] = df_amendements_matched["Similarité"]\
#.apply(lambda x:int(x.replace("%", "")))
#df_amendements_matched.Simi.hist(bins=100)
#del df_amendements_matched["Simi"]

In [ ]:
df_avis = pd.read_excel("tableau validé SP AN.xls", skiprows=[0])[["N°", "Avis SAJ titre I"]].fillna("")
df_avis["N°"] = df_avis["N°"].astype(str)
pd.read_excel("tableau validé SP AN.xls", skiprows=[0]).columns

In [ ]:
df_results_final = pd.merge(df_results, df_avis, left_on="N° v1", right_on="N°", how='left')
del df_results_final["N°"], df_results_final["article"]
df_results_final.to_csv("amendements_communs_AN_senat.csv", index=False)
df_results_final.to_excel("amendements_communs_AN_senat.xls", index=False)
df_results_final.head()
print(len(df_results_final))

# Matching with previous excel

In [ ]:
excel1 = pd.read_excel('ancien_derouleur.xlsx')
excel2 = pd.read_excel('nouveau_derouleur_a_remplir2.xlsx')

In [ ]:
def get_avis(df, nb):
    avis = df[df["N°"]==nb]["Avis SAJ "].values[0]
    return avis

def get_corresp_avis(corresp, nb):
    if nb in corresp:
        return get_avis(excel1, corresp[nb][0])
    return ""

def get_old_new_txt(corresp, nb):
    if nb in corresp:
        return (corresp[nb][1], corresp[nb][2], corresp[nb][3])
    return ("","","")

def get_text(df, nb):
    df_t = df[df.nb_2==nb]
    if len(df_t)>0:
        return df_t.txt.values[0]
    return ""

In [ ]:
avis, old_txt, new_txt, difference_txt, txt2= [], [], [], [], []
for i2, row2 in excel2.iterrows():
    nb2 = str(row2["N°"])
    avis.append(get_corresp_avis(corresp, nb2))
    extra_cols = get_old_new_txt(corresp, nb2)
    old_txt.append(extra_cols[0])
    new_txt.append(extra_cols[1])
    txt2.append(get_text(df_amendements_2, nb2))
    difference_txt.append(extra_cols[2])

In [ ]:
excel2["Avis SAJ"]=avis
excel2["Amendent v1"]=old_txt
excel2["Amendent v2"]=txt2
excel2["Similarité"]=difference_txt

In [ ]:
excel2["N°"] = excel2["N°"].apply(lambda x:str(x))
df_tmp = pd.DataFrame(excel2.groupby("txt_amendement")["N°"].agg(['count', "min"])).reset_index()


def has_similar(df_tmp, txt):
    if len(txt)<3:
        return ""
    if df_tmp[df_tmp["txt_amendement"]==txt]["count"].values[0] > 1:
        return 'Amendement similaire au ' + str( df_tmp[df_tmp["txt_amendement"]==txt]["min"].values[0])
    return ""

In [ ]:
similars = []
for i2, row2 in excel2.iterrows():
    similars.append(has_similar(df_tmp, row2.txt_amendement))

In [ ]:
excel2["Amendent similaire ?"]=similars

In [ ]:
excel2.to_excel('nouveau_derouleur_post_traitement.xlsx')